In [1]:
import numpy as np
import pandas as pd
import yaml

In [2]:
def open_result(dataset, experiment, moo):
    with open('cache/'+'_'.join([dataset, experiment, moo])+'.yaml', 'r') as f:
        doc = yaml.load(f, Loader=yaml.SafeLoader)
    return doc

def open_stats(dataset, experiment):
    for moo in ['MPC cost', 'MPC vs. #changes', 'feature']:
        with open('cache/'+'_'.join([dataset, experiment, moo])+'.yaml', 'r') as f:
            doc = yaml.load(f, Loader=yaml.SafeLoader)
            if not all([doc['Best solution rate'][key]==1 for key in doc['Best solution rate'] if key!='greedy']):
                print('Problem with '+dataset+'_'+experiment+'_'+moo+':')
                print([doc['Best solution rate'][key] for key in doc['Best solution rate'] if key!='greedy'])
                print()
            #assert all([doc['Best solution rate'][key]==1 for key in doc['Best solution rate'] if key!='greedy'])
            
    moo = 'MPC cost'
    with open('cache/'+'_'.join([dataset, experiment, moo])+'.yaml', 'r') as f:
        doc = yaml.load(f, Loader=yaml.SafeLoader)
    return doc['Feasible rate'], doc['Number of combinations']

In [3]:
datasets = ['german', 'giveme', 'taiwan', 'pima', 'student']

In [4]:
arrays = [[stat for stat in ['Feasible rate', 'Number of combinations']
          for experiment in ['logistic', 'tree', 'monotone-tree']],
          [experiment for stat in ['Feasible rate', 'Number of combinations']
          for experiment in ['logistic', 'tree', 'monotone-tree']],
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('info', 'experiment'))
stats = pd.DataFrame(columns=datasets, index=index)

In [5]:
for dataset in stats.columns:
    for info, experiment in stats.index:
        #print(dataset, experiment)
        try:
            frate, dim = open_stats(dataset, experiment)
            stats.loc[(info, experiment), dataset] = float(frate if info=='Feasible rate' else dim)
        except:
            pass

In [6]:
#pd.set_option('display.float_format', lambda x: '%.0e' % x if x>1000 else x)
stats

german         giveme  \
info                   experiment                                   
Feasible rate          logistic                  1              1   
                       tree                      1              1   
                       monotone-tree             1              1   
Number of combinations logistic       542575165440  2190677501250   
                       tree              267386880         907200   
                       monotone-tree     163577856         252000   

                                           taiwan     pima          student  
info                   experiment                                            
Feasible rate          logistic                 1        1                1  
                       tree                     1        1                1  
                       monotone-tree            1        1                1  
Number of combinations logistic       88155217920  1464100  471859200000000  
                       tree             228926250  9106944      96315899904  
                       monotone-tree    716601600   211680  471859200000000

## Table 1

In [7]:
table_1 = pd.DataFrame(columns=datasets, index=['mi-logistic', 'greedy', 'MAPOCAM'])

In [8]:
for dataset in table_1.columns:
    try:
        data_ = open_result(dataset, 'logistic', 'MPC cost')
        for method in table_1.index:
            #print(table_1.loc[method, dataset], data_['Time performance'][method])
            table_1.loc[method, dataset] = data_['Time performance'][method]
    except:
        pass

In [9]:
table_1.astype(float).round({'giveme': 3, 'german':2, 'taiwan':2,'pima':2,'student':2})

,german,giveme,taiwan,pima,student
mi-logistic,0.02,0.015,0.02,0.01,0.02
greedy,0.01,0.013,0.03,0.02,0.03
MAPOCAM,0.02,0.004,0.06,0.01,0.14


In [10]:
print(table_1.astype(float).round({'giveme': 3, 'german':2, 'taiwan':2,'pima':2,'student':2}).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  german &  giveme &  taiwan &  pima &  student \\
\midrule
mi-logistic &    0.02 &   0.015 &    0.02 &  0.01 &     0.02 \\
greedy      &    0.01 &   0.013 &    0.03 &  0.02 &     0.03 \\
MAPOCAM     &    0.02 &   0.004 &    0.06 &  0.01 &     0.14 \\
\bottomrule
\end{tabular}



## Table 2

In [11]:
arrays = [[moo for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-logistic', 'brute-force', 'MAPOCAM']],
          [method for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-logistic', 'brute-force', 'MAPOCAM']]
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('conflict', 'method'))
table_2 = pd.DataFrame(columns=datasets, index=index)

In [12]:
for dataset in table_2.columns:
    for conflict, method in table_2.index:
        try:
            data_ = open_result(dataset, 'logistic', conflict)
            table_2.loc[(conflict, method), dataset] = data_['Time performance'][method]
        except:
            pass

In [13]:
table_2.astype(float).round(2)

german  giveme  taiwan  pima  student
conflict         method                                               
MPC vs. #changes po-mi-logistic    0.01    0.01    0.02  0.01     0.02
                 brute-force       0.68    1.64    2.39  0.74     1.32
                 MAPOCAM           0.09    0.01    0.16  0.02     0.18
feature          po-mi-logistic    0.37    0.04    2.89  0.15     2.21
                 brute-force       0.46    1.07    2.35  0.56     1.43
                 MAPOCAM           0.15    0.01    0.45  0.03     0.48

In [14]:
print(table_2.astype(float).round(2).to_latex())

\begin{tabular}{llrrrrr}
\toprule
        &         &  german &  giveme &  taiwan &  pima &  student \\
conflict & method &         &         &         &       &          \\
\midrule
MPC vs. \#changes & po-mi-logistic &    0.01 &    0.01 &    0.02 &  0.01 &     0.02 \\
        & brute-force &    0.68 &    1.64 &    2.39 &  0.74 &     1.32 \\
        & MAPOCAM &    0.09 &    0.01 &    0.16 &  0.02 &     0.18 \\
feature & po-mi-logistic &    0.37 &    0.04 &    2.89 &  0.15 &     2.21 \\
        & brute-force &    0.46 &    1.07 &    2.35 &  0.56 &     1.43 \\
        & MAPOCAM &    0.15 &    0.01 &    0.45 &  0.03 &     0.48 \\
\bottomrule
\end{tabular}



## Table 3

In [15]:
arrays = [[moo for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P']]+
          [moo for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P']],
          [method for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P']]+
          [method for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P']]
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('conflict', 'method'))
table_3 = pd.DataFrame(columns=datasets, index=index)

In [16]:
for dataset in table_3.columns:
    for conflict, method in table_3.index:
        try:
            data_ = open_result(dataset, 'tree', conflict)
            table_3.loc[(conflict, method), dataset] = data_['Time performance'][method]
        except:
            pass

In [17]:
table_3.astype(float).round(2)

german  giveme  taiwan  pima  student
conflict         method                                            
MPC cost         mi-trees       0.02    0.02    0.04  0.02     0.02
                 greedy         0.01    0.01    0.02  0.02     0.01
                 MAPOCAM-N      0.05    0.02    5.42  0.56     0.02
                 MAPOCAM-P      0.04    0.01    0.49  0.12     0.01
MPC vs. #changes po-mi-trees    0.06    0.03    0.99  0.10     0.02
                 brute-force    0.67    0.21    2.41  1.45     0.62
                 MAPOCAM-N      0.23    0.02    0.73  0.21     0.04
                 MAPOCAM-P      0.18    0.01    0.13  0.09     0.02
feature          po-mi-trees    0.06    0.02    0.98  0.09     0.02
                 brute-force    0.24    0.13    1.70  1.01     0.15
                 MAPOCAM-N      0.15    0.06    1.28  0.45     0.05
                 MAPOCAM-P      0.17    0.02    0.29  0.18     0.03

In [18]:
print(table_3.astype(float).round(2).to_latex())

\begin{tabular}{llrrrrr}
\toprule
        &           &  german &  giveme &  taiwan &  pima &  student \\
conflict & method &         &         &         &       &          \\
\midrule
MPC cost & mi-trees &    0.02 &    0.02 &    0.04 &  0.02 &     0.02 \\
        & greedy &    0.01 &    0.01 &    0.02 &  0.02 &     0.01 \\
        & MAPOCAM-N &    0.05 &    0.02 &    5.42 &  0.56 &     0.02 \\
        & MAPOCAM-P &    0.04 &    0.01 &    0.49 &  0.12 &     0.01 \\
MPC vs. \#changes & po-mi-trees &    0.06 &    0.03 &    0.99 &  0.10 &     0.02 \\
        & brute-force &    0.67 &    0.21 &    2.41 &  1.45 &     0.62 \\
        & MAPOCAM-N &    0.23 &    0.02 &    0.73 &  0.21 &     0.04 \\
        & MAPOCAM-P &    0.18 &    0.01 &    0.13 &  0.09 &     0.02 \\
feature & po-mi-trees &    0.06 &    0.02 &    0.98 &  0.09 &     0.02 \\
        & brute-force &    0.24 &    0.13 &    1.70 &  1.01 &     0.15 \\
        & MAPOCAM-N &    0.15 &    0.06 &    1.28 &  0.45 &     0.05 \\
        

## Table 4

In [19]:
arrays = [[moo for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']]+
          [moo for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']],
          [method for moo in ['MPC cost']
           for method in ['mi-trees', 'greedy', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']]+
          [method for moo in ['MPC vs. #changes', 'feature']
           for method in ['po-mi-trees', 'brute-force', 'MAPOCAM-N', 'MAPOCAM-P', 'MAPOCAM-M']]
         ]
index = pd.MultiIndex.from_arrays(arrays, names=('conflict', 'method'))
table_4 = pd.DataFrame(columns=datasets, index=index)

In [20]:
for dataset in table_4.columns:
    for conflict, method in table_4.index:
        try:
            data_ = open_result(dataset, 'monotone-tree', conflict)
            table_4.loc[(conflict, method), dataset] = data_['Time performance'][method]
        except:
            pass

In [21]:
table_4.astype(float).round(2)

german  giveme  taiwan  pima  student
conflict         method                                            
MPC cost         mi-trees       0.02    0.03    0.07  0.03     0.02
                 greedy         0.01    0.02    0.08  0.03     0.01
                 MAPOCAM-N      0.69    0.01    1.35  0.24     0.10
                 MAPOCAM-P      0.02    0.01    0.05  0.02     0.01
                 MAPOCAM-M      0.02    0.01    0.05  0.02     0.01
MPC vs. #changes po-mi-trees    0.06    0.04    0.94  0.05     0.02
                 brute-force    1.35    0.68   20.94  0.75     0.46
                 MAPOCAM-N      0.23    0.01    0.56  0.12     0.11
                 MAPOCAM-P      0.04    0.01    0.08  0.02     0.02
                 MAPOCAM-M      0.04    0.02    0.09  0.02     0.01
feature          po-mi-trees    0.05    0.03    0.94  0.05     0.02
                 brute-force    0.50    0.46   17.04  0.53     0.27
                 MAPOCAM-N      0.23    0.07   12.56  0.22     0.22
                 MAPOCAM-P      0.04    0.01    0.94  0.03     0.01
                 MAPOCAM-M      0.05    0.02    0.90  0.03     0.01

## Colocar cache no calculo do objetivo (no greater than)!!!!

In [22]:
print(table_4.astype(float).round(2).to_latex())

\begin{tabular}{llrrrrr}
\toprule
        &           &  german &  giveme &  taiwan &  pima &  student \\
conflict & method &         &         &         &       &          \\
\midrule
MPC cost & mi-trees &    0.02 &    0.03 &    0.07 &  0.03 &     0.02 \\
        & greedy &    0.01 &    0.02 &    0.08 &  0.03 &     0.01 \\
        & MAPOCAM-N &    0.69 &    0.01 &    1.35 &  0.24 &     0.10 \\
        & MAPOCAM-P &    0.02 &    0.01 &    0.05 &  0.02 &     0.01 \\
        & MAPOCAM-M &    0.02 &    0.01 &    0.05 &  0.02 &     0.01 \\
MPC vs. \#changes & po-mi-trees &    0.06 &    0.04 &    0.94 &  0.05 &     0.02 \\
        & brute-force &    1.35 &    0.68 &   20.94 &  0.75 &     0.46 \\
        & MAPOCAM-N &    0.23 &    0.01 &    0.56 &  0.12 &     0.11 \\
        & MAPOCAM-P &    0.04 &    0.01 &    0.08 &  0.02 &     0.02 \\
        & MAPOCAM-M &    0.04 &    0.02 &    0.09 &  0.02 &     0.01 \\
feature & po-mi-trees &    0.05 &    0.03 &    0.94 &  0.05 &     0.02 \\
        & 